# Notebook for implementing ISCE2 interferometry to form Interferograms, and MintPy SBAS Time-Series analysis
This notebook will:
1. Find scenes from ASF Vertex using the asf_search library
2. Download said scenes
3. Use scenes to form interferograms using ISCE2
4. Utilize the closure phase to improve phase unwrapping
5. Create SBAS Time-Series of deformation using the closure phase corrected interferograms
6. Relate non-zero closure phase to surface vegatation and mositure changes using NDVI and EO moisture products
7. Report on erosion driver events and show relation to deformation and insar velocity from corrected SBAS time-series 

# To-DO
1. Get notebook working first with manually installed SLC images
2. Build off 'mintpy_get_isceburst.ipynb' to build interactive for retrieving SLCs

https://nbviewer.org/github/isce-framework/isce2-docs/blob/master/Notebooks/UNAVCO_2020/TOPS/topsApp.ipynb

# First, some helpful functions

In [115]:
# importing needed libraries for the entire notebook in one go
# %matplotlib inline
# %matplotlib widget
import os
from dateutil.parser import parse as parse_date
from datetime import datetime
import pandas as pd
import numpy as np
from osgeo import gdal, osr
import matplotlib.pyplot as plt
from typing import List, Union
from itertools import combinations
import eof
import geemap
import csv
import xml.etree.ElementTree as ET


# only need these two if you plan on searching andrequesting for data from within this notebook
# in my opinion, while it may take longer, it is much easier to use the ASF Vertex Website
# that said, if you are monitoring a reoccuring site and just need more imagery, searching using the notebook would be best
import asf_search as asf
import hyp3_sdk as sdk

In [116]:
def project_dir(proj_name):
    """
    This function reads in a string that you wish to make your working directory 
    for the InSAR project, and creates a data directory to store the data for ISCE2 and mintpy
    """

    #creates file on your desktop containing the work of this notebook
    work_dir = os.path.join(os.path.expanduser('~'),f'Desktop/{proj_name}')
    os.makedirs(work_dir, exist_ok=True)
    
    # file inside work_dir for isce2 interferometry
    if_dir = os.path.join(work_dir,'interferometry')
    os.makedirs(if_dir, exist_ok=True)
    
    # file inside work_dir for mintpy time-series
    ts_dir = os.path.join(work_dir,'time_series')
    os.makedirs(ts_dir, exist_ok=True)
    
    
    ######### ISCE2 Directories for interferometry
    xmls_dir = os.path.join(if_dir,'xmls')
    os.makedirs(xmls_dir, exist_ok=True)
    
    topsxml_dir = os.path.join(xmls_dir,'topsApp')
    os.makedirs(topsxml_dir, exist_ok=True)
    
    refxml_dir = os.path.join(xmls_dir,'reference')
    os.makedirs(refxml_dir, exist_ok=True)
    
    secxml_dir = os.path.join(xmls_dir,'secondary')
    os.makedirs(secxml_dir, exist_ok=True)
    
    xmldirectories = [topsxml_dir, refxml_dir, secxml_dir]
    
    isceref_dir = os.path.join(if_dir,'reference')
    os.makedirs(isceref_dir, exist_ok=True)
    
    iscesec_dir = os.path.join(if_dir,'secondary')
    os.makedirs(iscesec_dir, exist_ok=True)
    
    orbits_dir = os.path.join(if_dir,'orbits')
    os.makedirs(orbits_dir, exist_ok=True)
    
    ifdirectories = [isceref_dir, iscesec_dir, orbits_dir]
    
    ######### MinPy Directories for time-series
    baseline_dir = os.path.join(ts_dir,'baseline')
    os.makedirs(baseline_dir, exist_ok=True)
    
    ref_dir = os.path.join(ts_dir,'reference')
    os.makedirs(ref_dir, exist_ok=True)
    
    merged_dir = os.path.join(ts_dir,'merged')
    os.makedirs(merged_dir, exist_ok=True)
    
    geomref_dir = os.path.join(merged_dir,'geom_reference')
    os.makedirs(geomref_dir, exist_ok=True)
    
    interfer_dir = os.path.join(merged_dir,'interferograms')
    os.makedirs(interfer_dir, exist_ok=True)
    
    sec_dir = os.path.join(ts_dir,'secondaries')
    os.makedirs(sec_dir, exist_ok=True)

    mintpy_dir = os.path.join(ts_dir,'mintpy')
    os.makedirs(mintpy_dir, exist_ok=True)
    
    tsdirectories = [baseline_dir, ref_dir, merged_dir, geomref_dir, interfer_dir, sec_dir, mintpy_dir]

    return work_dir, ifdirectories, tsdirectories, xmldirectories

In [117]:
def get_triplets(slc_list):
    """
    this program will generate a dictionary with keys triplet_n, where n is the triplet stack
    each triplet_n contains 4 sets. Each set contains (path/to/ref.xml, path/to/sec.xml)
    
    slc_zips = list of directories containing the topsApp, reference, and secondary .xml files    
    created an returned using the project_dir fucntion
    """
    
    triplet_dict={}
    
    for i in range(len(slc_list) - 2):
        triplet_dict[f'triplet_{i+1}'] = ((slc_list[i],slc_list[i+1]), (slc_list[i+1],slc_list[i+2]), (slc_list[i+2], slc_list[i]))
        if i == 0:
            test_triplet = ((slc_list[i],slc_list[i+1]), (slc_list[i+1],slc_list[i+2]), (slc_list[i+2], slc_list[i]), (slc_list[i], slc_list[i+2]))
        else:
            continue
    return triplet_dict, test_triplet

In [118]:
# function to create the reference and secondary .xml files needed for ISCE2 interferometry
# function to write the 
def ref_sec_xml(slc_zips_list, slc_zips_dirs):
    for i in range(len(slc_zips_dirs)):
        for j, type in enumerate(['reference', 'secondary']):
            imset = ET.Element('component', name=type)
            safe = ET.SubElement(imset, 'property', name='safe').text = slc_zips_dirs[i]
            swaths = ET.SubElement(imset, 'property', name='swath number').text = str([1, 2])
            out_dir =ET.SubElement(imset, 'property', name='output directory').text = ifdirectories[j]
            orbit_dir =ET.SubElement(imset, 'property', name='orbit directory').text = ifdirectories[2]
            roi = ET.SubElement(imset, 'property', name='region of interest').text = str(isce_aoi)
            tree = ET.ElementTree(imset)
            tree.write(os.path.join(xmldirectories[j+1], f'{slc_zips_list[i][17:25]}{type[:3]}.xml'))

In [119]:
# function to create the topsApp.xml files for topsInSAR
def topsApp_xml(triplet_dict):
    for key in triplet_dict:
        for pair in triplet_dict[key]:
            data = ET.Element('topsApp')
            tinsar = ET.SubElement(data, 'component', name='topsinsar')
            snsr_name = ET.SubElement(tinsar, 'property', name='Sensor name').text = 'SENTINEL1'
            ref_xmlcomp = ET.SubElement(tinsar, 'component', name='reference')
            xml_file = ET.SubElement(ref_xmlcomp, 'catalog').text = os.path.join(xmldirectories[1], f"{pair[0][17:25]}ref.xml")
            sec_xmlcomp = ET.SubElement(tinsar, 'component', name='secondary')
            xml_file = ET.SubElement(sec_xmlcomp, 'catalog').text = os.path.join(xmldirectories[2], f"{pair[1][17:25]}sec.xml")
            azi_looks = ET.SubElement(tinsar, 'property', name='azimuth looks').text = str(6)
            range_looks = ET.SubElement(tinsar, 'property', name='range looks').text = str(2)
            
            phasefilt = ET.SubElement(tinsar, 'property', name='filter strength').text = str(0.2)
            
            unwrap_yn = ET.SubElement(tinsar, 'property', name='do unwrap').text = str(True) # or False
            unwrap_name = ET.SubElement(tinsar, 'property', name='unwrapper name').text = 'snaphu_mcf' # grass, icu, snaphu, snaphu_mcf, downsample_snap
            
            do_iono_corr = ET.SubElement(tinsar, 'property', name='do ionosphere correction').text = str(False)
            apply_iono_corr = ET.SubElement(tinsar, 'property', name='apply ionosphere correction').text = str(False)
            
            #choose from: ['subband', 'rawion', 'grd2ion', 'filt_gaussian', 'ionosphere_shift', 'ion2grd', 'esd']
            start_iono_corr = ET.SubElement(tinsar, 'property', name='start ionosphere step').text = 'filt_gaussian'
            end_iono_corr = ET.SubElement(tinsar, 'property', name='end ionosphere step').text = 'esd'
    
            geocode = ET.SubElement(tinsar, 'property', name='geocode bounding box').text = str(isce_aoi)
            
            tree = ET.ElementTree(data)
            tree.write(os.path.join(xmldirectories[0], f"{pair[0][17:25]}_{pair[1][17:25]}topsApp.xml"))

# Establish working directroy and data paths

In [120]:
# Establish working directories, and locate Sentinel-1 SLC .zip files
proj_name = 'Harris_etal_2023'
work_dir, ifdirectories, tsdirectories, xmldirectories = project_dir(proj_name)

In [121]:
# assuming you have downloaded .zip files covering your AOI from ASF Vertex
# enter the file directory below
slc_zips = '/home/wcc/Documents/InSAR/Brians_Paper/Harrisetal_slcs/2019_2020/'

slc_zips_list = sorted(os.listdir(slc_zips), key=lambda x: datetime.strptime(x[17:25], '%Y%m%d'))
slc_zips_dirs = [os.path.join(slc_zips, slc) for slc in slc_zips_list]
slc_zips_dates = [slc[17:25] for slc in slc_zips_list]

triplet_dict, test_triplet = get_triplets(slc_zips_list)

In [122]:
triplet_dict_length = len(triplet_dict)
display(triplet_dict_length)

21

In [123]:
# examine each triplet if you would like
# this shows the tuple that stores the string filenmem of the SLC .zip files
# in each triplet there will be 4 SLC pairs = 4 interferograms
# the first makes if_ij = the multilooked wrapped interferometric phase between SLC i and j
# the second makes if_jk = the multilooked wrapped interferometric phase between SLC j and k
# the third makes if_ki = the multilooked wrapped interferometric phase between SLC k and i
# where i, j, k are a time-series of SLC images
# choose a triplet to examine its pairs
# closure phase = if_ij+if_jk+if_ki

# triplet_choice = 'triple_1'
triplet_choice = f'triplet_{triplet_dict_length}'  # uncomment to just check the last triplet in the dict

if_ij = triplet_dict[triplet_choice][0]
if_jk = triplet_dict[triplet_choice][1]
if_ki = triplet_dict[triplet_choice][2]

print()
print(f'Pair 1:{if_ij}')
print(f'Pair 2:{if_jk}')
print(f'Pair 3:{if_ki}')


Pair 1:('S1A_IW_SLC__1SDV_20200901T001843_20200901T001910_034158_03F7BC_B964.zip', 'S1A_IW_SLC__1SDV_20200913T001843_20200913T001910_034333_03FDDC_3EF8.zip')
Pair 2:('S1A_IW_SLC__1SDV_20200913T001843_20200913T001910_034333_03FDDC_3EF8.zip', 'S1A_IW_SLC__1SDV_20200925T001843_20200925T001911_034508_040406_CA91.zip')
Pair 3:('S1A_IW_SLC__1SDV_20200925T001843_20200925T001911_034508_040406_CA91.zip', 'S1A_IW_SLC__1SDV_20200901T001843_20200901T001910_034158_03F7BC_B964.zip')


# Get bbox for your area of interest in the SLC images, faster processing with less data

In [124]:
## interactive map for you to draw a polygon to signify your aoi

## Create a map centered at a specific location
m = geemap.Map(center=[20, 0], zoom=2, basemap='HYBRID')

## Add drawing tools
m.add_draw_control()

## Display the map
display(m)

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…

In [125]:
# extract the vertices pf the feature
vertices = [(coords[0], coords[1]) for i, coords in enumerate(m.draw_features[0].getInfo()['geometry']['coordinates'][0])]

# create POLYGON string to use when searching asf for imagery
aoi = "POLYGON((" + ", ".join([f'{northing} {easting}' for northing, easting in vertices[:]]) +"))"

#create isce compatibile bbox
isce_aoi = [min([easting for northing, easting in vertices[:]]), max([easting for northing, easting in vertices[:]]), min([northing for northing, easting in vertices[:]]), max([northing for northing, easting in vertices[:]])]
print(isce_aoi)

[29.821583, 30.071471, -93.507385, -93.312378]


# Get s-1 orbit files (.EOF) using sentineleof library

In [105]:
#below downloads orbit files for your S1 SLC imagery to orbits_dir created with project_dir
for i,slc in enumerate(slc_zips_list):
    eof.download.download_eofs(
        orbit_dts=slc_zips_dates[i],  # slc date in str YYYYMMDD format
        missions=['S1A', 'S1B'],        # gets both S1 missions, third was just launched (2024) so may need updating
        sentinel_file=slc,              # image name
        save_dir=ifdirectories[2],      # orbits_dir
        orbit_type='precise'            # can be 'precise' or 'restituted'
    )

# Create Sentinel-1 SLC .xml files
- One reference and one secondary for each image

In [126]:
ref_sec_xml(slc_zips_list, slc_zips_dirs)

# Create topsApp.xml file
- one for each interferogram in each triplet
- (N_triplets * 4) - (N_triplets - 1)   ??

In [127]:
topsApp_xml(triplet_dict) # stores these in xmldirectories[0]

In [128]:
os.listdir(xmldirectories[0])

['20200820_20200727topsApp.xml',
 '20200210_20200317topsApp.xml',
 '20191013_20191106topsApp.xml',
 '20200913_20200820topsApp.xml',
 '20200808_20200715topsApp.xml',
 '20200329_20200210topsApp.xml',
 '20200913_20200925topsApp.xml',
 '20200808_20200820topsApp.xml',
 '20200703_20200609topsApp.xml',
 '20191212_20200117topsApp.xml',
 '20200129_20200210topsApp.xml',
 '20200609_20200621topsApp.xml',
 '20200925_20200901topsApp.xml',
 '20200703_20200715topsApp.xml',
 '20200410_20200422topsApp.xml',
 '20200621_20200703topsApp.xml',
 '20191130_20191106topsApp.xml',
 '20200117_20200129topsApp.xml',
 '20200210_20200117topsApp.xml',
 '20191130_20191212topsApp.xml',
 '20200410_20200317topsApp.xml',
 '20191001_20191013topsApp.xml',
 '20200422_20200609topsApp.xml',
 '20191118_20191130topsApp.xml',
 '20200317_20200329topsApp.xml',
 '20200901_20200913topsApp.xml',
 '20200727_20200808topsApp.xml',
 '20200609_20200410topsApp.xml',
 '20200715_20200621topsApp.xml',
 '20200129_20191212topsApp.xml',
 '20191118

In [130]:
testlist = os.listdir(xmldirectories[0])

In [129]:
topsappxml_list = sorted(os.listdir(xmldirectories[0]), key=lambda x: datetime.strptime(x[17:25], '%Y%m%d'))
topsappxml_list

ValueError: time data 'topsApp.' does not match format '%Y%m%d'

In [ ]:


triplet_dict={}

for i in range(len(slc_list) - 2):
    triplet_dict[f'triplet_{i+1}'] = ((slc_list[i],slc_list[i+1]), (slc_list[i+1],slc_list[i+2]), (slc_list[i+2], slc_list[i]))
    if i == 0:
        test_triplet = ((slc_list[i],slc_list[i+1]), (slc_list[i+1],slc_list[i+2]), (slc_list[i+2], slc_list[i]), (slc_list[i], slc_list[i+2]))
    else:
        continue
return triplet_dict, test_triplet

In [ ]:
topsApp_triplet_dict = 

# topsApp interferometry (finally!)